# Naive_Bayes
Filtro spam  
Mineração de emoções  
Separação de documentos  

In [1]:
import pandas as pd

In [2]:
import os.path
def path_base(base_name):
    current_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
    print(current_dir)
    data_dir = current_dir.replace('notebook','data')
    print(data_dir)
    data_base = data_dir + '\\' + base_name
    print(data_base)
    return data_base

In [3]:
base = pd.read_csv(path_base('db_dados_credito.csv'))

C:\MyPhyton\DataScience\notebook
C:\MyPhyton\DataScience\data
C:\MyPhyton\DataScience\data\db_dados_credito.csv


In [4]:
from pandas_profiling import ProfileReport
relatorio = ProfileReport(base)


In [5]:
relatorio.to_file('base_credit.html')

In [6]:
base.head(3)

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0


In [25]:
base.describe()

,clientid,income,age,loan,default
count,2000.000000,2000.000000,1997.000000,2000.000000,2000.000000
mean,1000.500000,45331.600018,40.807559,4444.369695,0.141500
std,577.494589,14326.327119,13.624469,3045.410024,0.348624
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,500.750000,32796.459717,28.990415,1939.708847,0.000000
50%,1000.500000,45789.117313,41.317159,3974.719419,0.000000
75%,1500.250000,57791.281668,52.587040,6432.410625,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000


# Tratamento de dados inconsistentes

In [26]:
base.loc[base['age']<=0,'age'] = base.age[base.age>0].mean()

In [27]:
print(base.loc[0].age)
print(base.loc[15].age)
print(base.loc[21].age)
print(base.loc[26].age)

59.017015066929204
40.92770044906149
40.92770044906149
40.92770044906149


## Previsores e classe

In [28]:
previsores = base.iloc[:,1:4]
classe = base.iloc[:,4]

# Valores nulos

In [29]:
import numpy as np
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')


In [30]:
imputer = imputer.fit(previsores.iloc[:, 0:3])
previsores.iloc[:, 0:3] = imputer.transform(previsores.iloc[:,0:3])

# Escalonamento de variáveis
Standardisation x = (x - média) / desvio

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [32]:
print(scaler.fit(previsores))
print(scaler.mean_)
print(scaler.fit_transform(previsores))
previsores = scaler.fit_transform(previsores)

StandardScaler(copy=True, with_mean=True, with_std=True)
[4.53316000e+04 4.09277004e+01 4.44436969e+03]
[[ 1.45393393  1.36538005  1.20281942]
 [-0.76217555  0.54265932  0.69642695]
 [ 0.83682073  1.67417101  1.17471147]
 ...
 [-0.07122592 -0.97448606  0.35420081]
 [-0.11000289  1.73936652 -0.92675625]
 [ 1.682986    1.14917551  0.96381038]]


In [33]:
type(previsores)

numpy.ndarray

# Divisão em base de treinos e testes

In [37]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.25, random_state=0)

In [38]:
print(previsores_treinamento.shape)
print(classe_treinamento.shape)

(1500, 3)
(1500,)


In [39]:
print(previsores_teste.shape)
print(classe_teste.shape)

(500, 3)
(500,)


In [40]:
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()
classificador.fit(previsores_treinamento, classe_treinamento)

GaussianNB(priors=None, var_smoothing=1e-09)

In [56]:
previsoes = classificador.predict(previsores_teste)
print(previsoes[0:10])
print(previsoes.shape)
print(classe_teste[0:10])
print(classe_teste.shape)

[1 0 0 0 0 0 0 0 0 0]
(500,)
405     1
1190    0
1132    0
731     0
1754    0
1178    0
1533    0
1303    0
1857    0
18      0
Name: default, dtype: int64
(500,)


In [57]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [59]:
precisao = accuracy_score(classe_teste,previsoes)
print(precisao)

0.938


In [60]:
matriz = confusion_matrix(classe_teste,previsoes)

In [61]:
print(matriz)

[[428   8]
 [ 23  41]]
